In [479]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import math
import re
import string
import sklearn
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
from bs4 import BeautifulSoup
import requests
%matplotlib notebook
import plotly.express as px ### for plotting the data on world map

In [480]:
path = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/05-25-2020.csv'
df = pd.read_csv(path)

In [481]:
df


,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
0,45001.0,Abbeville,South Carolina,US,2020-05-26 02:32:35,34.223334,-82.461707,35,0,0,35,"Abbeville, South Carolina, US"
1,22001.0,Acadia,Louisiana,US,2020-05-26 02:32:35,30.295065,-92.414197,299,18,0,281,"Acadia, Louisiana, US"
2,51001.0,Accomack,Virginia,US,2020-05-26 02:32:35,37.767072,-75.632346,731,11,0,720,"Accomack, Virginia, US"
3,16001.0,Ada,Idaho,US,2020-05-26 02:32:35,43.452658,-116.241552,796,23,0,773,"Ada, Idaho, US"
4,19001.0,Adair,Iowa,US,2020-05-26 02:32:35,41.330756,-94.471059,8,0,0,8,"Adair, Iowa, US"
...,...,...,...,...,...,...,...,...,...,...,...,...
3430,NaN,NaN,Terengganu,Malaysia,2020-05-26 02:32:35,5.311700,103.132400,113,1,110,0,"Terengganu, Malaysia"
3431,NaN,NaN,W.P. Kuala Lumpur,Malaysia,2020-05-26 02:32:35,3.139000,101.686900,1803,17,1292,0,"W.P. Kuala Lumpur, Malaysia"
3432,NaN,NaN,W.P. Labuan,Malaysia,2020-05-26 02:32:35,5.283100,115.230800,16,0,16,0,"W.P. Labuan, Malaysia"
3433,NaN,NaN,W.P. Putrajaya,Malaysia,2020-05-26 02:32:35,2.926400,101.696400,93,1,83,0,"W.P. Putrajaya, Malaysia"


In [482]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3435 entries, 0 to 3434
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   FIPS            3008 non-null   float64
 1   Admin2          3011 non-null   object 
 2   Province_State  3255 non-null   object 
 3   Country_Region  3435 non-null   object 
 4   Last_Update     3435 non-null   object 
 5   Lat             3364 non-null   float64
 6   Long_           3364 non-null   float64
 7   Confirmed       3435 non-null   int64  
 8   Deaths          3435 non-null   int64  
 9   Recovered       3435 non-null   int64  
 10  Active          3435 non-null   int64  
 11  Combined_Key    3435 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 322.2+ KB


In [483]:
df.shape

(3435, 12)

In [484]:
df.size

41220

In [485]:
df.count

<bound method DataFrame.count of          FIPS     Admin2  ... Active                   Combined_Key
0     45001.0  Abbeville  ...     35  Abbeville, South Carolina, US
1     22001.0     Acadia  ...    281          Acadia, Louisiana, US
2     51001.0   Accomack  ...    720         Accomack, Virginia, US
3     16001.0        Ada  ...    773                 Ada, Idaho, US
4     19001.0      Adair  ...      8                Adair, Iowa, US
...       ...        ...  ...    ...                            ...
3430      NaN        NaN  ...      0           Terengganu, Malaysia
3431      NaN        NaN  ...      0    W.P. Kuala Lumpur, Malaysia
3432      NaN        NaN  ...      0          W.P. Labuan, Malaysia
3433      NaN        NaN  ...      0       W.P. Putrajaya, Malaysia
3434      NaN        NaN  ...      0              Unknown, Malaysia

[3435 rows x 12 columns]>

In [486]:
df.nunique()

FIPS              3007
Admin2            1751
Province_State     288
Country_Region     191
Last_Update          2
Lat               3362
Long_             3358
Confirmed          996
Deaths             331
Recovered          297
Active             917
Combined_Key      3435
dtype: int64

In [487]:
df.describe()

,FIPS,Lat,Long_,Confirmed,Deaths,Recovered,Active
count,3008.000000,3364.000000,3364.000000,3435.000000,3435.000000,3435.000000,3435.000000
mean,31342.178191,35.918724,-79.649223,1600.648326,105.024454,649.717322,858.606114
std,17187.857574,12.075505,42.420865,11309.374837,1048.660831,8695.547609,10411.615891
min,66.000000,-52.368000,-172.104600,0.000000,0.000000,0.000000,-345610.000000
25%,18160.500000,33.618192,-96.573857,12.000000,0.000000,0.000000,9.000000
50%,29164.000000,37.854779,-88.048863,52.000000,1.000000,0.000000,43.000000
75%,46065.500000,41.590908,-80.744866,280.000000,11.000000,0.000000,219.500000
max,99999.000000,71.706900,178.065000,353427.000000,36914.000000,379157.000000,230996.000000


**preprossing**

removing  null values

In [488]:
print (df.isna().sum(),"\n")
print('Null Values',df.isna().sum().sum())

FIPS              427
Admin2            424
Province_State    180
Country_Region      0
Last_Update         0
Lat                71
Long_              71
Confirmed           0
Deaths              0
Recovered           0
Active              0
Combined_Key        0
dtype: int64 

Null Values 1173


In [489]:
df['FIPS'] = df['FIPS'].replace(np.nan, 0)
df['Province_State'] = df['Province_State'].replace(np.nan, 0)
df['Admin2'] = df['Admin2'].replace(np.nan, 0)
df['Lat'] = df['Lat'].replace(np.nan, 0)
df['Long_'] = df['Long_'].replace(np.nan, 0)


In [490]:
df.isnull().sum()

FIPS              0
Admin2            0
Province_State    0
Country_Region    0
Last_Update       0
Lat               0
Long_             0
Confirmed         0
Deaths            0
Recovered         0
Active            0
Combined_Key      0
dtype: int64

In [491]:
df.drop(['FIPS', 'Admin2','Last_Update','Province_State', 'Combined_Key'], axis=1, inplace=True)

In [492]:
df.rename(columns={'Country_Region': "Country"}, inplace=True)
df.head()

,Country,Lat,Long_,Confirmed,Deaths,Recovered,Active
0,US,34.223334,-82.461707,35,0,0,35
1,US,30.295065,-92.414197,299,18,0,281
2,US,37.767072,-75.632346,731,11,0,720
3,US,43.452658,-116.241552,796,23,0,773
4,US,41.330756,-94.471059,8,0,0,8


label encoding 

In [493]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Country'] = le.fit_transform(df['Country'])
df

,Country,Lat,Long_,Confirmed,Deaths,Recovered,Active
0,178,34.223334,-82.461707,35,0,0,35
1,178,30.295065,-92.414197,299,18,0,281
2,178,37.767072,-75.632346,731,11,0,720
3,178,43.452658,-116.241552,796,23,0,773
4,178,41.330756,-94.471059,8,0,0,8
...,...,...,...,...,...,...,...
3430,108,5.311700,103.132400,113,1,110,0
3431,108,3.139000,101.686900,1803,17,1292,0
3432,108,5.283100,115.230800,16,0,16,0
3433,108,2.926400,101.696400,93,1,83,0


In [494]:
world = df.groupby("Country")['Confirmed','Active','Recovered','Deaths'].sum().reset_index()
world.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,Country,Confirmed,Active,Recovered,Deaths
0,0,11180,9857,1097,223
1,1,1004,177,795,32
2,2,8503,3147,4747,609
3,3,763,49,663,51
4,4,70,48,18,4
